In [1]:
import hashlib
import pandas as pd
import numpy as np
import requests
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pyquery import PyQuery
from kerastuner.tuners import RandomSearch

In [ ]:
!pip install pyquery

In [ ]:
# dataset = pd.read_csv("k.csv")
# features = dataset[["ball_1","ball_2","ball_3","ball_4","ball_5","ball_6","ball_7","ball_8","ball_9","ball_10"]].values
# labels = dataset[["result"]].values

# print(len(features), len(labels))

In [2]:
hashes = []

r = requests.get("https://csgopolygon.gg/rolls.php")
html = PyQuery(r.text)

for tr in html.find("tr").items():
    seed = tr("td").eq(1)
    lottery = tr("td").eq(2)
    rolls = tr("td").eq(3)


    if seed.text() != "SERVER SEED IN USE" and len(seed.text()) > 1:
        rolls =  str(rolls.text())
        rolls = rolls.split("-")
        roll_start = rolls[0]
        roll_finish = rolls[1]

        hashes.append({"hash" : seed.text(),"lottery" : int(lottery.text()),"roll_start" : int(roll_start),"roll_finish" : int(roll_finish)})

rolls = []
split_counter = 0
for i in range(0,len(hashes)):
    for z in range(hashes[i]["roll_start"],hashes[i]["roll_finish"]):
        string = "{}-{}-{}".format(hashes[i]["hash"], hashes[i]["lottery"], z)
        hashed = hashlib.sha256(str(string).encode("utf-8")).hexdigest()
        hashed = int(hashed[:8], 16)
        rolled = hashed % 15

        rolls.append(rolled)

split_counter = 0
features = []
labels = []
slice_counter = 0
for i in range(0, len(rolls)):
    if i >= 10:
        features.append(rolls[slice_counter:i])
        labels.append([rolls[i]])
        slice_counter += 1

In [3]:
def new_split(quantity,features):
    splitted_features = []
    splitted_labels = []
    slice_counter = 0
    for i in range(0,len(features)):
        if i % quantity == 0 and i > 0:
            splitted_features.append(features[slice_counter:i])
            splitted_labels.append([features[i]])
            slice_counter += quantity
    return (splitted_features, splitted_labels)

In [4]:
def split_dataset(features,labels):
    splitted_features = []
    splitted_labels = []
    slice_counter = 0
    for i in range(0,len(features)):
        if i % 10 == 0 and i > 0:
            splitted_features.append(features[slice_counter:i])
            splitted_labels.append(labels[i - 1])
            slice_counter += 10
    return (splitted_features,splitted_labels)

In [5]:
(splitted_features,splitted_labels) = split_dataset(features,labels)

In [6]:
splitted_features[2]

[[10, 9, 2, 2, 0, 8, 0, 2, 9, 6],
 [9, 2, 2, 0, 8, 0, 2, 9, 6, 2],
 [2, 2, 0, 8, 0, 2, 9, 6, 2, 1],
 [2, 0, 8, 0, 2, 9, 6, 2, 1, 3],
 [0, 8, 0, 2, 9, 6, 2, 1, 3, 6],
 [8, 0, 2, 9, 6, 2, 1, 3, 6, 11],
 [0, 2, 9, 6, 2, 1, 3, 6, 11, 10],
 [2, 9, 6, 2, 1, 3, 6, 11, 10, 11],
 [9, 6, 2, 1, 3, 6, 11, 10, 11, 5],
 [6, 2, 1, 3, 6, 11, 10, 11, 5, 14]]

In [7]:
splitted_labels[1]

[6]

In [8]:
# other wat encoding
result_encoded = []
for label in splitted_labels:
    if label[0] == 0:
        result_encoded.append([1,0,0])
    elif label[0] < 8:
        result_encoded.append([0,1,0])
    elif label[0] >= 8:
        result_encoded.append([0,0,1])
    else:
        print("confused {}".format(label[0]))
features = splitted_features
labels = result_encoded
print(len(features), len(labels))

329071 329071


In [9]:
labels[4]

[0, 0, 1]

In [10]:
#features = np.array(features[:-1])
features = np.array(features)
labels = np.array(labels)

In [11]:
features.shape

(329071, 10, 10)

In [12]:
# scaler = StandardScaler()
# features = scaler.fit_transform(features)
# labels = scaler.fit_transform(labels)

In [13]:
train_features, test_features, train_labels, test_labels = train_test_split(features,labels, test_size=0.1, random_state=1)

In [18]:
train_labels.shape

(296163, 3)

In [15]:
train_labels[1]

array([0, 0, 1])

In [22]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(10, input_shape=(10,10)))
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=1024,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return model

In [23]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(train_features, train_labels,
             epochs=30,
             batch_size=8096,
             validation_data=(test_features, test_labels))

INFO:tensorflow:Reloading Oracle from existing project my_dir\helloworld\oracle.json
Train on 296163 samples, validate on 32908 samples
Epoch 1/30
145728/296163 [=============>................] - ETA: 16s - loss: 2.2783 - acc: 0.64 - ETA: 13s - loss: 2.5579 - acc: 0.64 - ETA: 12s - loss: 4.4682 - acc: 0.55 - ETA: 11s - loss: 4.0889 - acc: 0.53 - ETA: 11s - loss: 4.2261 - acc: 0.55 - ETA: 11s - loss: 4.3631 - acc: 0.57 - ETA: 11s - loss: 4.3852 - acc: 0.58 - ETA: 10s - loss: 4.2947 - acc: 0.58 - ETA: 10s - loss: 4.1303 - acc: 0.59 - ETA: 10s - loss: 4.0309 - acc: 0.57 - ETA: 9s - loss: 3.9392 - acc: 0.5559 - ETA: 9s - loss: 3.8429 - acc: 0.563 - ETA: 8s - loss: 3.7762 - acc: 0.569 - ETA: 8s - loss: 3.7233 - acc: 0.574 - ETA: 8s - loss: 3.6710 - acc: 0.579 - ETA: 7s - loss: 3.6146 - acc: 0.583 - ETA: 7s - loss: 3.5662 - acc: 0.584 - ETA: 7s - loss: 3.5205 - acc: 0.5735

KeyboardInterrupt: 

In [ ]:
#model.fit(train_features,train_labels, epochs=38, batch_size=1024)

In [ ]:
model.evaluate(test_features, test_labels)

In [ ]:
np.argmax(model.predict(np.array([[
    [12,5,4,3,11,13,10,5,1,1],
    [5,4,3,11,13,10,5,1,1,8],
    [4,3,11,13,10,5,1,1,8,13],
    [3,11,13,10,5,1,1,8,13,3],
    [11,13,10,5,1,1,8,13,3,10],
    [13,10,5,1,1,8,13,3,10,11],
    [10,5,1,1,8,13,3,10,11,3],
    [5,1,1,8,13,3,10,11,3,11],
    [1,1,8,13,3,10,11,3,11,13],
    [1,8,13,3,10,11,3,11,13,3]
]])))

In [ ]:
model.save("model.h5")